# Tmax Subset

A notebook to subset Tmax daily for the 13000 GHS urban areas to identify dates >40c, consecuritve days >40 c etc.

### Depdencies

In [1]:
import pandas as pd
import numpy as np
import xarray as xr
from random import random
from itertools import groupby
from operator import itemgetter
import geopandas as gpd 
import glob

In [5]:
def csv_to_xr(file_in, time_dim, space_dim):
    
    """ Function reads in a csv w/ GHS-UCDB IDs and temp, isolates the temp
    and returns a xarray data array with dims set to city ids and dates
    
    Args:
        file_in = file name and path
        time_dim = name for time dim as a str ... use date :-)
        space_dim = col name for GHS-UCDB IDs as an str (ID_HDC_G0)
    """
    
    df = pd.read_csv(file_in) # read the file in as a df
    print(df.shape)
    
    df_id = df[space_dim] # get IDs
    df_temp = df.iloc[:,3:] # get only temp columns
    df_temp.index = df_id # set index values
    df_temp_drop = df_temp.dropna() # Drop cities w/ no temp record 
    print(len(df_temp_drop))
    
    temp_np = df_temp_drop.to_numpy() # turn temp cols into an np array
    
    # make xr Data Array w/ data as temp and dims as spece (e.g. id)
    temp_xr_da = xr.DataArray(temp_np, coords=[df_temp_drop.index, df_temp_drop.columns], dims=[space_dim, time_dim])
    
    return temp_xr_da

In [ ]:
def temp_eventTot(xarray, Tthresh, year):
    """ Function returns the number of days within a year where Tmax > Tthresh for each city.
    
    Args: 
        xarray = an xarray object with dims = (space, times)
        Tthresh = int of temp threshold
    """
    
    ## NOTE FOR SOME REASON out.ID_HDC_G0 cannot be fed a string ... note sure why so be careful with col names
    out = xarray.where(xarray > Tthresh, drop = True)
    id_list = []
    event_tot = []
    df_out = pd.DataFrame()
    
    for index, loc in enumerate(out.ID_HDC_G0):
        id_list.append(out.ID_HDC_G0.values[index])
        event_tot.append(len(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').date.values))
    
    df_out['ID_HDC_G0'] = id_list
    df_out[year] = event_tot
    
    return df_out

In [ ]:
def event_loop(dir_nm, time_dim, space_dim, Tthresh):
    
    """ Loop through a dir with csvs to calc the total number of events greater than a threshold.
        Leap years explain the difference in shapes 368 vs 369
    
    Args:
        dir_nm = dir path to loop through
        time_dim = name for time dim as a str ... use date :-) for csv_to_xr function
        space_dim = col name for GHS-UCDB IDs as an str (ID_HDC_G0) for csv_to_xr function
        Tthresh = int of temp threshold for temp_event function -- 40.6 is
    """
    
    # Open the GHS-ID List with GeoPANDAS read_file
    ghs_ids_fn = 'GHS-UCSB-IDS.csv'
    ghs_ids_df = pd.read_csv(DATA_INTERIM+ghs_ids_fn)
    
    # Git File list
    fn_list = glob.glob(DAILY_PATH+'*.csv')
    
    for fn in sorted(fn_list):
        
        # Get year for arg for temp_event function
        year = fn.split('GHS-Tmax-DAILY_')[1].split('.csv')[0]
        print(year)
        
        temp_xr_da = csv_to_xr(fn, time_dim, space_dim)
        
        df_out = temp_eventTot(temp_xr_da, Tthresh, year)
        
        ghs_ids_df = ghs_ids_df.merge(df_out, on='ID_HDC_G0', how = 'outer') #<<<<----- NEED TO FIX THIS
    
    # build in later drop all NA GHS-IDs
    
    return ghs_ids_df


In [ ]:
def temp_eventL(xarray, Tthresh, year):
    """ Function calculates the length of each Tmax threshold event as the number of days in a row
    greater than a threshold within a year where Tmax > Tthresh for each city.
    
    Args: 
        xarray = an xarray object with dims = (space, times)
        Tthresh = int of temp threshold
    """
    
    ## NOTE FOR SOME REASON out.ID_HDC_G0 cannot be fed a string ... note sure why so be careful with col names
    out = xarray.where(xarray > Tthresh, drop = True)
    id_list = []
    event_L = []
    df_out = pd.DataFrame()
    
    for index, loc in enumerate(out.ID_HDC_G0):
        id_list.append(out.ID_HDC_G0.values[index])
        event_tot.append(len(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').date.values))
    
    df_out['ID_HDC_G0'] = id_list
    df_out[year] = event_tot
    
    return df_out

In [2]:
# File Paths
DAILY_PATH = '/home/cascade/projects/data_out/CHIRTS-GHS-DAILY/'
DATA_INTERIM = '/home/cascade/projects/UrbanHeat/data/interim/'
DATA_OUT = '/home/cascade/projects/data_out/'

In [3]:
fn_in = 'GHS-Tmax-DAILY_1983.csv'


In [7]:
xr1993 = csv_to_xr(DAILY_PATH+fn_in, 'date', 'ID_HDC_G0')

(13135, 368)
13067


In [9]:
xr1993

<xarray.DataArray (ID_HDC_G0: 13067, date: 365)>
array([[-43.921947, -33.71345 , -33.054974, ..., -12.416152, -13.232986,
        -15.403823],
       [ -4.804248,  -3.914425,  -7.533999, ...,  -5.186461, -10.945722,
        -16.29516 ],
       [-23.904118, -17.422953, -13.182008, ..., -12.788978, -11.337886,
        -10.00939 ],
       ...,
       [ 16.028023,  17.73603 ,  20.493294, ...,  14.559421,  15.160739,
         15.184024],
       [ 16.420553,  17.87142 ,  22.519674, ...,  15.680964,  16.169733,
         16.039179],
       [ 16.6943  ,  17.559229,  21.480919, ...,  14.446052,  15.235602,
         14.005591]])
Coordinates:
  * ID_HDC_G0  (ID_HDC_G0) int64 5782 3316 5645 3185 ... 1116 1114 1161 1169
  * date       (date) object '1983.01.01' '1983.01.02' ... '1983.12.31'

In [12]:
out = xr1993.where(xr1993 > 40.6, drop = True)
out

<xarray.DataArray (ID_HDC_G0: 4483, date: 315)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * ID_HDC_G0  (ID_HDC_G0) int64 2784 2372 6156 2833 ... 1077 12977 13012 13020
  * date       (date) object '1983.01.04' '1983.01.05' ... '1983.12.29'

In [21]:
#event_tot.append(len(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').date.values))

arr = []
for index, loc in enumerate(out.ID_HDC_G0):
    print(out.ID_HDC_G0.values[index])
    x = (out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').date.values) #
    arr.append(x)

2784
['1983.07.27']
2372
['1983.07.31']
6156
['1983.07.31']
2833
['1983.07.27']
2885
['1983.07.27']
2367
['1983.07.31']
2334
['1983.07.31']
2331
['1983.07.31']
2258
['1983.07.31']
2952
['1983.07.28']
6414
['1983.07.31']
5066
['1983.07.12']
2275
['1983.07.27']
2965
['1983.07.27' '1983.07.29']
2280
['1983.07.27']
5884
['1983.07.16' '1983.07.28']
2186
['1983.07.31']
2862
['1983.07.27' '1983.07.28']
6564
['1983.07.31']
2264
['1983.07.26' '1983.07.27']
2529
['1983.07.21']
2504
['1983.07.21']
2559
['1983.07.21']
2964
['1983.07.20']
2538
['1983.07.21']
2586
['1983.07.21' '1983.07.29']
2246
['1983.07.26' '1983.07.27']
6181
['1983.07.30' '1983.07.31']
5928
['1983.07.16' '1983.07.17' '1983.07.28' '1983.07.29']
5950
['1983.06.23' '1983.07.16' '1983.07.17' '1983.07.28' '1983.07.29'
 '1983.07.30' '1983.08.22']
6411
['1983.07.31']
171
['1983.08.06' '1983.08.07' '1983.08.08']
166
['1983.08.06' '1983.08.07' '1983.08.08']
5649
['1983.07.13' '1983.07.14' '1983.07.15' '1983.07.16' '1983.08.15']
5974
['19

['1983.06.20' '1983.06.21' '1983.06.22' '1983.06.23' '1983.06.24'
 '1983.06.25' '1983.06.26' '1983.06.27' '1983.06.28' '1983.06.30'
 '1983.07.17' '1983.07.21' '1983.07.22' '1983.07.23' '1983.07.24'
 '1983.07.25' '1983.07.28' '1983.07.29' '1983.07.30' '1983.07.31'
 '1983.08.01' '1983.08.04' '1983.08.05']
5912
['1983.07.27' '1983.07.29']
5224
['1983.07.16' '1983.07.17' '1983.07.18']
5816
['1983.07.13' '1983.07.14']
5810
['1983.07.13' '1983.07.14' '1983.08.15']
5939
['1983.06.12' '1983.06.13' '1983.06.18' '1983.06.19' '1983.06.20'
 '1983.06.21' '1983.06.22' '1983.06.23' '1983.06.26' '1983.07.06'
 '1983.07.07' '1983.07.10' '1983.07.11' '1983.07.12' '1983.07.13'
 '1983.07.14' '1983.07.15' '1983.07.16' '1983.07.17' '1983.07.18'
 '1983.07.20' '1983.07.21' '1983.07.22' '1983.07.23' '1983.07.24'
 '1983.07.25' '1983.07.26' '1983.07.27' '1983.07.28' '1983.07.29'
 '1983.07.30' '1983.07.31' '1983.08.01' '1983.08.03' '1983.08.13'
 '1983.08.14' '1983.08.15' '1983.08.16']
4494
['1983.07.12']
4672
['19

['1983.07.15' '1983.07.16' '1983.07.19' '1983.08.14' '1983.08.15']
2024
['1983.07.19' '1983.08.07']
4658
['1983.07.01' '1983.07.02' '1983.07.10' '1983.07.11' '1983.07.12'
 '1983.07.17' '1983.07.18']
29
['1983.08.06' '1983.08.07']
5907
['1983.06.19' '1983.06.20' '1983.06.21' '1983.06.22' '1983.06.23'
 '1983.06.25' '1983.06.26' '1983.06.27' '1983.07.06' '1983.07.07'
 '1983.07.08' '1983.07.15' '1983.07.16' '1983.07.17' '1983.07.18'
 '1983.07.19' '1983.07.20' '1983.07.21' '1983.07.22' '1983.07.23'
 '1983.07.24' '1983.07.25' '1983.07.26' '1983.07.27' '1983.07.28'
 '1983.07.29' '1983.07.30' '1983.07.31' '1983.08.01' '1983.08.03'
 '1983.08.13' '1983.08.14' '1983.08.15' '1983.08.16']
5920
['1983.07.27' '1983.07.28' '1983.07.29' '1983.07.30']
5913
['1983.06.23' '1983.07.08' '1983.07.17' '1983.07.20' '1983.07.21'
 '1983.07.23' '1983.07.24' '1983.07.27' '1983.07.28' '1983.07.29'
 '1983.07.30' '1983.08.15' '1983.08.16']
5941
['1983.07.23' '1983.07.27' '1983.07.28' '1983.07.29']
5957
['1983.06.22' 

['1983.08.06']
1661
['1983.06.15' '1983.08.05']
12841
['1983.08.06']
6312
['1983.06.23' '1983.06.24' '1983.06.26' '1983.06.27' '1983.06.28'
 '1983.06.29' '1983.06.30' '1983.08.01']
6205
['1983.06.29' '1983.06.30']
6203
['1983.06.22' '1983.06.23' '1983.06.24' '1983.06.26' '1983.06.27'
 '1983.06.28' '1983.06.29' '1983.06.30' '1983.07.01' '1983.07.21'
 '1983.07.22' '1983.07.23' '1983.08.01']
1613
['1983.06.15']
2761
['1983.06.27' '1983.07.07' '1983.07.23' '1983.07.24' '1983.07.25'
 '1983.07.31' '1983.08.09' '1983.08.29' '1983.08.30']
2790
['1983.07.07' '1983.07.24' '1983.08.29']
5349
['1983.06.01' '1983.06.02' '1983.06.03' '1983.06.04' '1983.06.05'
 '1983.06.06' '1983.06.10' '1983.06.11' '1983.06.15' '1983.06.16'
 '1983.06.17' '1983.06.18' '1983.06.19' '1983.06.20' '1983.06.21'
 '1983.06.24' '1983.06.25' '1983.06.30' '1983.07.01' '1983.07.02'
 '1983.07.03' '1983.07.04' '1983.07.05' '1983.07.06' '1983.07.07'
 '1983.07.08' '1983.07.09' '1983.07.10' '1983.07.11' '1983.07.12'
 '1983.07.13' '1

['1983.06.21' '1983.06.22' '1983.06.23' '1983.06.24' '1983.06.25'
 '1983.06.26' '1983.06.27' '1983.06.28' '1983.06.29' '1983.06.30'
 '1983.07.01' '1983.07.02' '1983.08.01']
5332
['1983.05.28' '1983.06.01' '1983.06.02' '1983.06.03' '1983.06.04'
 '1983.06.05' '1983.06.06' '1983.06.10' '1983.06.11' '1983.06.15'
 '1983.06.16' '1983.06.17' '1983.06.18' '1983.06.19' '1983.06.20'
 '1983.06.21' '1983.06.24' '1983.06.25' '1983.06.30' '1983.07.01'
 '1983.07.02' '1983.07.03' '1983.07.04' '1983.07.05' '1983.07.06'
 '1983.07.07' '1983.07.08' '1983.07.09' '1983.07.10' '1983.07.11'
 '1983.07.12' '1983.07.13' '1983.07.14' '1983.07.15' '1983.07.16'
 '1983.07.17' '1983.07.18' '1983.07.19' '1983.07.20' '1983.07.21'
 '1983.07.22' '1983.07.23' '1983.07.24' '1983.07.25' '1983.07.26'
 '1983.07.27' '1983.07.28' '1983.07.29' '1983.07.30' '1983.07.31'
 '1983.08.01' '1983.08.02' '1983.08.03' '1983.08.04' '1983.08.05'
 '1983.08.06' '1983.08.07' '1983.08.08' '1983.08.09' '1983.08.10'
 '1983.08.11' '1983.08.12' '19

...

['1983.05.01' '1983.05.04' '1983.05.05']
5556
['1983.05.12' '1983.05.13' '1983.05.14' '1983.06.07' '1983.06.11'
 '1983.06.12' '1983.06.13' '1983.06.14' '1983.06.15' '1983.06.16'
 '1983.06.17' '1983.06.19' '1983.06.20' '1983.06.21' '1983.06.22'
 '1983.06.23' '1983.06.24' '1983.06.25' '1983.06.26' '1983.06.27'
 '1983.06.29' '1983.06.30' '1983.07.01' '1983.07.02' '1983.07.03'
 '1983.07.04' '1983.07.05' '1983.07.06' '1983.07.07' '1983.07.08'
 '1983.07.09' '1983.07.10' '1983.07.11' '1983.07.12' '1983.07.13'
 '1983.07.14' '1983.07.15' '1983.07.16' '1983.07.17' '1983.07.18'
 '1983.07.19' '1983.07.20' '1983.07.21' '1983.07.22' '1983.07.23'
 '1983.07.24' '1983.07.26' '1983.07.27' '1983.07.28' '1983.07.29'
 '1983.07.30' '1983.08.01' '1983.08.02' '1983.08.03' '1983.08.04'
 '1983.08.05' '1983.08.06' '1983.08.07' '1983.08.08' '1983.08.09'
 '1983.08.13' '1983.08.14' '1983.08.18' '1983.08.21' '1983.08.26'
 '1983.08.27' '1983.08.28' '1983.08.29' '1983.08.30' '1983.08.31'
 '1983.09.07' '1983.09.25']
86

['1983.04.05' '1983.04.17' '1983.04.20' '1983.04.21' '1983.04.24'
 '1983.04.27' '1983.04.28' '1983.04.29' '1983.04.30' '1983.05.01'
 '1983.05.02' '1983.05.03' '1983.05.04' '1983.05.05' '1983.05.06'
 '1983.05.07' '1983.05.09' '1983.05.11']
8289
['1983.05.03' '1983.05.04' '1983.05.05' '1983.05.06' '1983.05.07'
 '1983.05.14' '1983.05.24' '1983.05.31' '1983.06.03' '1983.06.04'
 '1983.06.05' '1983.06.06' '1983.07.04']
1501
['1983.03.06' '1983.04.06']
1766
['1983.03.26' '1983.04.06' '1983.04.08' '1983.04.09' '1983.04.10'
 '1983.04.11' '1983.04.29' '1983.04.30' '1983.05.01' '1983.05.09']
3593
['1983.04.06' '1983.04.07' '1983.04.28' '1983.04.29' '1983.05.01'
 '1983.05.02' '1983.05.03' '1983.05.04' '1983.05.05' '1983.05.06'
 '1983.05.07' '1983.05.08' '1983.05.09' '1983.05.10' '1983.05.17'
 '1983.05.25' '1983.05.26' '1983.05.27' '1983.05.28' '1983.05.29'
 '1983.05.30' '1983.06.02']
3099
['1983.03.27' '1983.03.28' '1983.04.03' '1983.04.04' '1983.04.05'
 '1983.04.06' '1983.04.07' '1983.04.08' '198

['1983.05.03' '1983.05.07' '1983.05.08' '1983.05.09' '1983.05.13'
 '1983.05.14' '1983.05.15' '1983.06.02' '1983.06.03' '1983.06.04'
 '1983.06.28' '1983.07.04']
3180
['1983.03.19' '1983.03.24' '1983.03.25' '1983.03.26' '1983.03.27'
 '1983.03.28' '1983.03.29' '1983.04.02' '1983.04.03' '1983.04.04'
 '1983.04.05' '1983.04.06' '1983.04.07' '1983.04.08' '1983.04.16'
 '1983.04.17' '1983.04.18' '1983.04.20' '1983.04.21' '1983.04.22'
 '1983.04.23' '1983.04.24' '1983.04.25' '1983.04.26' '1983.04.27'
 '1983.04.28' '1983.04.29' '1983.04.30' '1983.05.01' '1983.05.02'
 '1983.05.03' '1983.05.04' '1983.05.05' '1983.05.06' '1983.05.07'
 '1983.05.08' '1983.05.09' '1983.05.12' '1983.05.29']
1687
['1983.03.26' '1983.04.30']
2340
['1983.04.28' '1983.04.29']
1539
['1983.03.09' '1983.03.14' '1983.03.15' '1983.03.21' '1983.03.26'
 '1983.03.27' '1983.03.28' '1983.03.29' '1983.03.30' '1983.04.06'
 '1983.04.07' '1983.04.08' '1983.04.09' '1983.04.10' '1983.04.30']
8508
['1983.05.03' '1983.05.07' '1983.05.08' '198

['1983.03.15' '1983.03.16' '1983.03.18' '1983.03.19' '1983.03.22'
 '1983.03.26' '1983.03.28' '1983.03.29' '1983.03.30' '1983.03.31'
 '1983.04.05' '1983.04.06' '1983.04.07' '1983.04.09' '1983.04.14'
 '1983.04.15' '1983.04.16' '1983.04.18' '1983.04.19' '1983.04.20'
 '1983.04.21' '1983.04.22' '1983.04.23' '1983.04.24' '1983.04.25'
 '1983.04.26' '1983.04.27' '1983.04.29' '1983.05.02' '1983.05.03']
3018
['1983.03.19' '1983.03.20' '1983.03.24' '1983.03.25' '1983.03.26'
 '1983.04.05' '1983.04.15' '1983.04.16' '1983.04.19' '1983.04.20'
 '1983.04.21' '1983.04.22' '1983.04.23' '1983.04.24' '1983.04.25'
 '1983.04.26' '1983.04.27' '1983.04.28' '1983.04.29' '1983.04.30'
 '1983.05.01' '1983.05.03' '1983.05.04' '1983.05.05' '1983.05.12']
3352
['1983.03.04' '1983.03.05' '1983.03.06' '1983.03.10' '1983.03.12'
 '1983.03.15' '1983.03.16' '1983.03.17' '1983.03.18' '1983.03.19'
 '1983.03.20' '1983.03.21' '1983.03.23' '1983.03.25' '1983.03.26'
 '1983.03.27' '1983.03.28' '1983.03.29' '1983.04.02' '1983.04.03

['1983.02.13' '1983.02.27' '1983.02.28' '1983.04.02' '1983.04.03'
 '1983.04.04' '1983.04.05' '1983.04.06']
12676
['1983.04.28' '1983.06.11' '1983.10.07']
12655
['1983.05.24' '1983.11.08' '1983.12.08']
3979
['1983.03.15']
3963
['1983.03.15']
12912
['1983.02.28' '1983.04.20']
12899
['1983.03.13']
4219
['1983.02.14']
12907
['1983.03.21']
3650
['1983.01.26' '1983.05.23' '1983.11.13' '1983.12.06' '1983.12.10'
 '1983.12.20']
798
['1983.11.05']
4244
['1983.11.10' '1983.11.11' '1983.11.12' '1983.11.13' '1983.11.14'
 '1983.11.15']
4258
['1983.10.31' '1983.11.09' '1983.11.10' '1983.11.11' '1983.11.12'
 '1983.11.13' '1983.11.14' '1983.11.15']
4197
['1983.11.13']
1414
['1983.02.17' '1983.05.17']
1410
['1983.02.17' '1983.05.17']
4472
['1983.02.03' '1983.10.31' '1983.11.09' '1983.11.10' '1983.11.11'
 '1983.11.12' '1983.11.13' '1983.11.21' '1983.11.22' '1983.11.23']
4939
['1983.11.13']
4457
['1983.01.11' '1983.02.03' '1983.03.04' '1983.03.06' '1983.09.25'
 '1983.10.30' '1983.10.31' '1983.11.09' '1983

In [ ]:
# Run routine
# all_events_df = event_loop(DAILY_PATH, 'date', 'ID_HDC_G0', 40.6)

In [ ]:
all_events_df.head()

In [ ]:
# Move IDS to Index 

all_events_df = all_events_df.set_index(['ID_HDC_G0', 'CTR_MN_NM'], drop = True)
all_events_df.head()

In [ ]:
# Drop NaNs
all_events_df_drop = all_events_df.dropna(how = 'all')
all_events_df_drop.shape

In [ ]:
all_events_df_drop.head()

In [ ]:
df_out = all_events_df_drop.copy()

In [ ]:
df_out['ID_HDC_G0'] = all_events_df_drop.index

In [ ]:
df_out.head()

In [ ]:
all_events_df_drop = all_events_df_drop.reset_index()

In [ ]:
all_events_df_drop.head()

In [ ]:
#all_events_df_drop.to_csv(DATA_OUT+'20190831_TMax-GHS_TotEvents83-2016.csv')

In [ ]:
india = all_events_df_drop[all_events_df_drop['CTR_MN_NM'] == 'India']

In [ ]:
india.head()

### Plot it

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import figure
%matplotlib inline

figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
y = range(1,125)
year = '2010'
country = 'INDIA'
plt.hist(india[year], bins = 125)
plt.xlabel('Number of Days in '+year+' where Tmax >40c in ')
plt.ylabel('Number of cities')
plt.title(country+': For all cities with Tmax >40, how many days in '+year+' were >40C? ')

In [ ]:
# MAP BACK TO POLYGONS AND LOOK AT IT 
SHP_DIR = '/Users/cascade/Github/UrbanHeat/data/raw/ghs-ucdb/'
shp_fn = 'GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_0.shp'
shps = gpd.read_file(SHP_DIR+shp_fn)

In [ ]:
df_ghs = gpd.GeoDataFrame()
df_ghs['geometry'] = shps.geometry
df_ghs['ID_HDC_G0'] = shps.ID_HDC_G0

In [ ]:
df_merge = df_ghs.merge(events, on='ID_HDC_G0', how = 'inner') #<<<<----- NEED TO FIX THIS

In [ ]:
# Write it out
DATA_INTERIM = '/Users/cascade/Github/UrbanHeat/data/interim/'
fn_out = 'GHS-TmaxDaily-events.shp'
df_merge.to_file(DATA_INTERIM+fn_out)

# Old Code

In [ ]:
# This will return the ID and Date where Tmax is greater than 40 as a dict, but will not return actual tempatures 

Tmax = np.random.randint(20, high=50, size=(3,10)) # Make a 3x10 random list
print(Tmax)
results = np.where(Tmax > 40) # find the index and rows
coords = list(zip(results[0], results[1])) # zip the i and js into tuples

b = [(k, list(list(zip(*g))[1])) for k, g in groupby(coords, itemgetter(0))] # group by rows

print(b)
dict_out = dict(b) # turn into a dict, where keys are city ids and values are dates
dict_out


In [ ]:
for key, value in dict_out.items():
    print(key, value)

In [ ]:
np.where(Tmax > 40, Tmax, Tmax*0) 

In [ ]:
np.where(Tmax > 40) 

In [ ]:
np.argwhere(Tmax>1)

In [ ]:
def temp_search(array):
    results = np.where(array > 40) # find the index and rows
    coords = list(zip(results[0], results[1])) # zip the i and js into tuples
    b = [(k, list(list(zip(*g))[1])) for k, g in groupby(coords, itemgetter(0))] # group by rows
    dict_out = dict(b) # turn into a dict, where keys are city ids and values are dates

    return dict_out

In [ ]:
file_in = '/Users/cascade/Desktop/GHS-Tmax-DAILY_1983.csv'

df = pd.read_csv(file_in)

In [ ]:
df.head()

In [ ]:
df_sub = df.iloc[:,3:]

In [ ]:
df_sub_drop = df.dropna(how='all')

In [ ]:
df_sub.head()
arr = df_sub.to_numpy()

In [ ]:
arr.shape

In [ ]:
np.unique(arr)

In [ ]:
tmax_search = temp_search(arr)

In [ ]:
# Make some fake data
Tmax = np.random.randint(20, high=50, size=(3,10))
locs = ['001', '002', '003']
times = pd.date_range('2000-01-01', periods=10)

In [ ]:
foo = xr.DataArray(Tmax, coords=[locs, times], dims=['space', 'times'])

In [ ]:
foo

In [ ]:
out = foo.where(foo > 40, drop = True)
out

In [ ]:
for loc in out.space:
    print(len(out.sel(space = loc).dropna(dim = 'times').times.values))

In [ ]:
for x in out.space.values:
    print(x)

In [ ]:
out.space.values[0]